# ASHRAE - Great Energy Predictor Challenge

The following notebook builds a LightGBM model for predicting energy comsumption of buildings across various sites using past recorded hourly steam, chilled, electric meter readings. 

The notebook consits of the following section

1. Reading the data
2. Data Preprocessing
3. Feature Engineering
4. Validating the LightGBM  model
5. Scoring the test data

In [79]:
import tqdm
import gc
import os
import random
import lightgbm as lgb
import numpy as np
import pandas as pd
import datetime
from tqdm import tqdm 
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from meteocalc import feels_like, Temp
from sklearn.model_selection import StratifiedKFold

path_train = "train.csv"
path_test = "test.csv"
path_building = "building_metadata.csv"
path_weather_train = "weather_train.csv"
path_weather_test = "weather_test.csv"

seed = 0
random.seed(seed)

## Reading train data
Reading train data along with building and weather metadata.

In [55]:
df_train = pd.read_csv(path_train)
building = pd.read_csv(path_building)
le = LabelEncoder()
building.primary_use = le.fit_transform(building.primary_use)
weather_train = pd.read_csv(path_weather_train)

In [56]:
df_train.shape

(20216100, 4)

In [57]:
## Memory optimization

# Original code from https://www.kaggle.com/gemartin/load-data-reduce-memory-usage by @gemartin
# Modified to support timestamp type, categorical type
# Modified to add option to use float16

from pandas.api.types import is_datetime64_any_dtype as is_datetime
from pandas.api.types import is_categorical_dtype

def reduce_mem_usage(df, use_float16=False):
    """
    Iterate through all the columns of a dataframe and modify the data type to reduce memory usage.        
    """
    
    start_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage of dataframe is {:.2f} MB".format(start_mem))
    
    for col in df.columns:
        if is_datetime(df[col]) or is_categorical_dtype(df[col]):
            continue
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if use_float16 and c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype("category")

    end_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage after optimization is: {:.2f} MB".format(end_mem))
    print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [58]:
df_train = reduce_mem_usage(df_train, use_float16=True)
building = reduce_mem_usage(building, use_float16=True)

Memory usage of dataframe is 616.95 MB
Memory usage after optimization is: 173.90 MB
Decreased by 71.8%
Memory usage of dataframe is 0.07 MB
Memory usage after optimization is: 0.02 MB
Decreased by 74.9%


In [59]:
rowstodrop = pd.read_csv('rows_to_drop.csv')

In [60]:
df_train.drop(rowstodrop.loc[:, '0'], inplace=True)
df_train.reset_index(drop=True, inplace=True)

In [61]:
df_train.shape

(19229841, 4)

## Data Preprocessing
There are two files with features that need to be merged with the data. One is building metadata that has information on the buildings and the other is weather data that has information on the weather. 

In [62]:
# Add new Features
weather_train["datetime"] = pd.to_datetime(weather_train["timestamp"])
weather_train["day"] = weather_train["datetime"].dt.day
weather_train["week"] = weather_train["datetime"].dt.week
weather_train["month"] = weather_train["datetime"].dt.month
    
# Reset Index for Fast Update
weather_train = weather_train.set_index(['site_id','day','month'])

air_temperature_filler = pd.DataFrame(weather_train.groupby(['site_id','day','month'])['air_temperature'].mean(),columns=["air_temperature"])
weather_train.update(air_temperature_filler,overwrite=False)

# Step 1
cloud_coverage_filler = weather_train.groupby(['site_id','day','month'])['cloud_coverage'].mean()
# Step 2
cloud_coverage_filler = pd.DataFrame(cloud_coverage_filler.fillna(method='ffill'),columns=["cloud_coverage"])

weather_train.update(cloud_coverage_filler,overwrite=False)

due_temperature_filler = pd.DataFrame(weather_train.groupby(['site_id','day','month'])['dew_temperature'].mean(),columns=["dew_temperature"])
weather_train.update(due_temperature_filler,overwrite=False)

# Step 1
sea_level_filler = weather_train.groupby(['site_id','day','month'])['sea_level_pressure'].mean()
# Step 2
sea_level_filler = pd.DataFrame(sea_level_filler.fillna(method='ffill'),columns=['sea_level_pressure'])

weather_train.update(sea_level_filler,overwrite=False)

wind_direction_filler =  pd.DataFrame(weather_train.groupby(['site_id','day','month'])['wind_direction'].mean(),columns=['wind_direction'])
weather_train.update(wind_direction_filler,overwrite=False)

wind_speed_filler =  pd.DataFrame(weather_train.groupby(['site_id','day','month'])['wind_speed'].mean(),columns=['wind_speed'])
weather_train.update(wind_speed_filler,overwrite=False)

# Step 1
precip_depth_filler = weather_train.groupby(['site_id','day','month'])['precip_depth_1_hr'].mean()
# Step 2
precip_depth_filler = pd.DataFrame(precip_depth_filler.fillna(method='ffill'),columns=['precip_depth_1_hr'])

weather_train.update(precip_depth_filler,overwrite=False)

weather_train = weather_train.reset_index()
weather_train = weather_train.drop(['datetime','day','week','month'],axis=1)

# Feature Engineering

In [63]:
def timestampalign(df):
    """
    Aligning timestamp for weather data based on UTC offsets of estimated site locations.
    """
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df.loc[(df['site_id']==0) | (df['site_id']==3) | (df['site_id']==6) | (df['site_id']==7) | (df['site_id']==8) | (df['site_id']==11) | (df['site_id']==14) | (df['site_id']==15), 'timestamp'] = df['timestamp'] - pd.Timedelta(4, unit='h')
    df.loc[(df['site_id']==2) | (df['site_id']==4) | (df['site_id']==10), 'timestamp'] = df['timestamp'] - pd.Timedelta(7, unit='h')
    df.loc[(df['site_id']==9) | (df['site_id']==13), 'timestamp'] = df['timestamp'] - pd.Timedelta(5, unit='h')

def prepare_data(X, building_data, weather_data, test=False):
    """
    Preparing final dataset with all features.
    """
    X.timestamp = pd.to_datetime(X.timestamp, format="%Y-%m-%d %H:%M:%S")
    
    #Filling NaNs using interpolation
    #weather_data = weather_data.groupby("site_id").apply(lambda group: group.interpolate(limit_direction="both"))
    
    weather_data['relative_humidity'] = 100 * (np.exp((17.625 * weather_data['dew_temperature']) / (243.04 + weather_data['dew_temperature']))
                                     / np.exp((17.625 * weather_data['air_temperature'])/(243.04 + weather_data['air_temperature'])))
    
    def calculate_fl(df):
            flike_final = []
            flike = []
            # calculate Feels Like temperature
            for i in range(len(df)):
                at = df['air_temperature'][i]
                rh = df['relative_humidity'][i]
                ws = df['wind_speed'][i]
                flike.append(feels_like(Temp(at, unit = 'C'), rh, ws))
            for i in range(len(flike)):
                flike_final.append(flike[i].f)
            df['feels_like'] = flike_final
            del flike_final, flike, at, rh, ws
    calculate_fl(weather_data)
    
    X = X.merge(building_data, on="building_id", how="left")
    X = X.merge(weather_data, on=["site_id", "timestamp"], how="left")
    
    X.sort_values("timestamp")
    X.reset_index(drop=True)
    
    gc.collect()
    
#     holidays = ["2016-01-01", "2016-01-18", "2016-02-15", "2016-05-30", "2016-07-04",
#                 "2016-09-05", "2016-10-10", "2016-11-11", "2016-11-24", "2016-12-26",
#                 "2017-01-02", "2017-01-16", "2017-02-20", "2017-05-29", "2017-07-04",
#                 "2017-09-04", "2017-10-09", "2017-11-10", "2017-11-23", "2017-12-25",
#                 "2018-01-01", "2018-01-15", "2018-02-19", "2018-05-28", "2018-07-04",
#                 "2018-09-03", "2018-10-08", "2018-11-12", "2018-11-22", "2018-12-25",
#                 "2019-01-01"]
    
    X.square_feet = np.log1p(X.square_feet)
    
    X["hour"] = X.timestamp.dt.hour
    X["weekday"] = X.timestamp.dt.weekday
    X["month"] = X.timestamp.dt.month
    X['month'].replace((1,2,3), 1, inplace=True)
    X['month'].replace((4,5,6), 2, inplace=True)
    X['month'].replace((7,8,9), 3, inplace=True)
    X['month'].replace((10,11,12), 4, inplace=True)
    #X["is_holiday"] = (X.timestamp.isin(holidays)).astype(int)
    #X.loc[(X['weekday'] == 5) | (X['weekday'] == 6), 'isWeekend'] = 1
    #X['isWeekend'] = X['isWeekend'].fillna(0)
    
    drop_features = ["timestamp"]

    if test:
        row_ids = X.row_id
        X.drop("row_id", axis=1, inplace=True)
        X.drop(drop_features, axis=1, inplace=True)
        return X, row_ids
    else:
        X = X.query('not (building_id <= 104 & meter == 0 & timestamp <= "2016-05-20")')
        X = X.query('not (building_id == 1099 & meter == 2 & meter_reading > 3e4)')
        y = np.log1p(X.meter_reading)
        X.drop("meter_reading", axis=1, inplace=True)
        X.drop(drop_features, axis=1, inplace=True)
        return X, y

In [64]:
timestampalign(weather_train)

In [65]:
X_train, y_train = prepare_data(df_train, building, weather_train)

del df_train, weather_train
gc.collect()

84

In [69]:
X_train.head()

,building_id,meter,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,relative_humidity,feels_like,hour,weekday,month
1,105,0,1,0,10.832181,NaN,5.0,3.8,0.0,2.4,0.0,1020.9,240.0,3.1,90.575533,36.933764,0,4,1
2,106,0,1,0,8.589514,NaN,4.0,3.8,0.0,2.4,0.0,1020.9,240.0,3.1,90.575533,36.933764,0,4,1
3,106,3,1,0,8.589514,NaN,4.0,3.8,0.0,2.4,0.0,1020.9,240.0,3.1,90.575533,36.933764,0,4,1
4,107,0,1,0,11.487946,2005.0,10.0,3.8,0.0,2.4,0.0,1020.9,240.0,3.1,90.575533,36.933764,0,4,1
5,108,0,1,0,11.309352,1913.0,5.0,3.8,0.0,2.4,0.0,1020.9,240.0,3.1,90.575533,36.933764,0,4,1


## Validating the LightGBM model

In [66]:
categorical_features = ["building_id", "site_id", "meter", "primary_use", "weekday"]
features = [c for c in X_train.columns if c not in ['month']]

params = {
    "objective": "regression",
    "boosting": "gbdt",
    "num_leaves": 1580,
    "learning_rate": 0.05,
    "feature_fraction": 0.85,
    "reg_lambda": 1,
    "metric": "rmse",
}

kf = StratifiedKFold(n_splits = 5, shuffle=False, random_state = 0)
models = []
for train_idx, val_idx in kf.split(X_train, X_train['month']):
    train_feats = X_train[features].loc[train_idx]
    train_target = y_train.loc[train_idx]
    val_feats = X_train[features].loc[val_idx]
    val_target = y_train.loc[val_idx]
    
    lgb_train = lgb.Dataset(train_feats, label=train_target, categorical_feature=categorical_features, free_raw_data=False)
    lgb_val = lgb.Dataset(val_feats, label=val_target, categorical_feature=categorical_features, free_raw_data=False)
    
    model = lgb.train(params, train_set=lgb_train, num_boost_round=3000, valid_sets=[lgb_train, lgb_val], verbose_eval=25,
                     early_stopping_rounds = 50)
    models.append(model)
    del train_feats, train_target, val_feats, val_target, lgb_train, lgb_val
    gc.collect()

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
C:\Anaconda3\lib\site-packages\lightgbm\basic.py:1243: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 50 rounds
[25]	training's rmse: 0.961825	valid_1's rmse: 1.04926
[50]	training's rmse: 0.726986	valid_1's rmse: 0.859611
[75]	training's rmse: 0.664619	valid_1's rmse: 0.822081
[100]	training's rmse: 0.63348	valid_1's rmse: 0.812239
[125]	training's rmse: 0.611409	valid_1's rmse: 0.808279
[150]	training's rmse: 0.596548	valid_1's rmse: 0.805918
[175]	training's rmse: 0.585239	valid_1's rmse: 0.804875
[200]	training's rmse: 0.576228	valid_1's rmse: 0.804335
[225]	training's rmse: 0.569297	valid_1's rmse: 0.804328
[250]	training's rmse: 0.563411	valid_1's rmse: 0.804159
[275]	training's rmse: 0.557315	valid_1's rmse: 0.80394
[300]	training's rmse: 0.552406	valid_1's rmse: 0.804042
[325]	training's rmse: 0.547756	valid_1's rmse: 0.804058
Early stopping, best iteration is:
[277]	training's rmse: 0.556716	valid_1's rmse: 0.803898


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike


Training until validation scores don't improve for 50 rounds
[25]	training's rmse: 0.9779	valid_1's rmse: 0.988205
[50]	training's rmse: 0.747706	valid_1's rmse: 0.776744
[75]	training's rmse: 0.684132	valid_1's rmse: 0.730255
[100]	training's rmse: 0.650051	valid_1's rmse: 0.717574
[125]	training's rmse: 0.626643	valid_1's rmse: 0.712329
[150]	training's rmse: 0.61018	valid_1's rmse: 0.709716
[175]	training's rmse: 0.59771	valid_1's rmse: 0.70826
[200]	training's rmse: 0.588754	valid_1's rmse: 0.707705
[225]	training's rmse: 0.58086	valid_1's rmse: 0.707239
[250]	training's rmse: 0.574301	valid_1's rmse: 0.706963
[275]	training's rmse: 0.568396	valid_1's rmse: 0.70683
[300]	training's rmse: 0.562994	valid_1's rmse: 0.706825
[325]	training's rmse: 0.55838	valid_1's rmse: 0.706894
Early stopping, best iteration is:
[295]	training's rmse: 0.564113	valid_1's rmse: 0.706776
Training until validation scores don't improve for 50 rounds
[25]	training's rmse: 0.979476	valid_1's rmse: 0.969597


## Preparing test data
Preparing test data with same features as train data.

In [68]:
df_test = pd.read_csv(path_test)
weather_test = pd.read_csv(path_weather_test)

In [72]:
 # Add new Features
weather_test["datetime"] = pd.to_datetime(weather_test["timestamp"])
weather_test["day"] = weather_test["datetime"].dt.day
weather_test["week"] = weather_test["datetime"].dt.week
weather_test["month"] = weather_test["datetime"].dt.month
    
    # Reset Index for Fast Update
weather_test = weather_test.set_index(['site_id','day','month'])

air_temperature_filler = pd.DataFrame(weather_test.groupby(['site_id','day','month'])['air_temperature'].mean(),columns=["air_temperature"])
weather_test.update(air_temperature_filler,overwrite=False)

    # Step 1
cloud_coverage_filler = weather_test.groupby(['site_id','day','month'])['cloud_coverage'].mean()
    # Step 2
cloud_coverage_filler = pd.DataFrame(cloud_coverage_filler.fillna(method='ffill'),columns=["cloud_coverage"])

weather_test.update(cloud_coverage_filler,overwrite=False)

due_temperature_filler = pd.DataFrame(weather_test.groupby(['site_id','day','month'])['dew_temperature'].mean(),columns=["dew_temperature"])
weather_test.update(due_temperature_filler,overwrite=False)

    # Step 1
sea_level_filler = weather_test.groupby(['site_id','day','month'])['sea_level_pressure'].mean()
    # Step 2
sea_level_filler = pd.DataFrame(sea_level_filler.fillna(method='ffill'),columns=['sea_level_pressure'])

weather_test.update(sea_level_filler,overwrite=False)

wind_direction_filler =  pd.DataFrame(weather_test.groupby(['site_id','day','month'])['wind_direction'].mean(),columns=['wind_direction'])
weather_test.update(wind_direction_filler,overwrite=False)

wind_speed_filler =  pd.DataFrame(weather_test.groupby(['site_id','day','month'])['wind_speed'].mean(),columns=['wind_speed'])
weather_test.update(wind_speed_filler,overwrite=False)

    # Step 1
precip_depth_filler = weather_test.groupby(['site_id','day','month'])['precip_depth_1_hr'].mean()
    # Step 2
precip_depth_filler = pd.DataFrame(precip_depth_filler.fillna(method='ffill'),columns=['precip_depth_1_hr'])

weather_test.update(precip_depth_filler,overwrite=False)

weather_test = weather_test.reset_index()
weather_test = weather_test.drop(['datetime','day','week','month'],axis=1)

In [73]:
timestampalign(weather_test)
df_test = reduce_mem_usage(df_test)
X_test, row_ids = prepare_data(df_test, building, weather_test, test=True)

Memory usage of dataframe is 1272.51 MB
Memory usage after optimization is: 358.65 MB
Decreased by 71.8%


In [77]:
del df_test, building, weather_test
gc.collect()

233

## Scoring test data
Averaging predictions from the five fold models.

In [84]:
iterations = 120
set_size = len(X_test)
batch_size = set_size // iterations
pred = []
for i in tqdm(range(iterations)):
    pos = i*batch_size
    fold_preds = [np.expm1(model.predict(X_test[features].iloc[pos : pos+batch_size])) for model in models]
    pred.extend(np.mean(fold_preds, axis=0))

print(len(pred))

100%|█████████████████████████████████████████████████████████████████████████████| 120/120 [7:20:31<00:00, 220.26s/it]


41697600


## Submission
Preparing final file for submission.

In [85]:
submission = pd.DataFrame({"row_id": row_ids, "meter_reading": np.clip(pred, 0, a_max=None)})
submission.to_csv("submissionashraemodel14.csv.gz", index=False, compression='gzip')